#Import data and select population and TAZ ID

In [ ]:
import pandas as pd

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %cd "/content/drive/MyDrive/CS"
# %ls 

In [ ]:
#Data link
#https://drive.google.com/file/d/1N0Bvhuh-0LQL2VQO70mE0dlGp9zK1Msl/view?usp=sharing

In [ ]:
df1 = pd.read_csv("lat_lon_3miles_TAZ.csv")
#df1.head()
#df1.info()

In [ ]:
df2=df1[['OBJECTID','CityArea','TAZID','YEAR2023', 'lat_','lon_']]
df=df2[df2.YEAR2023 > 30]

#Function of destination choose

Chooses random locations based on their population weights.

In [ ]:
import random

def choose_locations(n):
  # Create a list of weights based on the value of YEAR2023 for each observation
  weights = list(df['YEAR2023'])

  # Get the integer TAZIDs as a list
  tazids = df['TAZID'].tolist()

  # Choose n locations randomly, considering the population weight for each location
  chosen_tazids = random.choices(tazids, weights=weights, k=n)

  # Print the chosen TAZIDs
  print('chosen_tazids=',chosen_tazids)
  print('')

  # Subset the rows in the DataFrame that correspond to the selected TAZIDs, and extract lat_ and lon_
  lat_lon_pairs = []
  for tazid in chosen_tazids:
    row = df.loc[df['TAZID'] == tazid]
    lat = row['lat_'].values[0]
    lon = row['lon_'].values[0]
    lat_lon_pairs.append((lat, lon))

  # Print the chosen (lat, lon) pairs
  #print(lat_lon_pairs)
  return chosen_tazids, lat_lon_pairs

#Function of read API and get travel time,distance and slope for bicycle mode

In [ ]:
#create a function called get_route_info that takes two inputs: the start point and end point as tuples of latitude and longitude. 

import requests
import json
import math


def get_route_info(start_point, end_point, profile):
    # Specify the API endpoint
    url = "https://api.openrouteservice.org/v2/directions/{profile}"


    params = {
        "api_key": "5b3ce3597851110001cf624879353a62e64d48bfbfd041d761fba9bd",
        "start": f"{start_point[1]},{start_point[0]}",
        "end": f"{end_point[1]},{end_point[0]}",
        "profile": profile,
        "preference": "fastest",
    }


    # Send the API request
    response = requests.get(url.format(profile=params["profile"]), params=params)

    data = response.json()
    #print(data)

    # Extract the travel time and travel distance
    travel_time = data["features"][0]["properties"]["segments"][0]["duration"]
    travel_distance = data["features"][0]["properties"]["segments"][0]["distance"]

    # Extract the elevation data and calculate the average slope
    elevation_data = data["features"][0]["geometry"]["coordinates"]
    elevation_gain = 0
    elevation_down = 0
    distance = 0
    for i in range(len(elevation_data)-1):
        # Calculate the distance between two points using the Haversine formula
        lon1, lat1 = elevation_data[i]
        lon2, lat2 = elevation_data[i+1]
        R = 6371  # Radius of the earth in km
        dLat = (lat2-lat1) * (math.pi/180)
        dLon = (lon2-lon1) * (math.pi/180)
        a = math.sin(dLat/2) * math.sin(dLat/2) + \
            math.cos((lat1) * (math.pi/180)) * math.cos((lat2) * (math.pi/180)) * \
            math.sin(dLon/2) * math.sin(dLon/2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        distance += R * c * 1000

        # Calculate the elevation gain between two points
        elevation1 = elevation_data[i][1]
        elevation2 = elevation_data[i+1][1]
        if elevation2 > elevation1:
            elevation_gain += elevation2 - elevation1
        else:
            elevation_down += elevation1 - elevation2

    # Calculate the average slope
    slope = elevation_gain / distance

    return travel_time, travel_distance, slope, elevation_gain, elevation_down, data

#start = (41.75312339358314, -111.83486333648153)
#start = (41.740988, -111.815286)  # Richards Hall, Logan, Utah
#end = (41.745764, -111.809146)  # Aggie Village Community Center, Logan, Utah
#travel_time, travel_distance, elevation_gain = get_route_info(start, end)

# Print the results
#print("Travel time: {} seconds".format(travel_time))
#print("Travel distance: {} meters".format(travel_distance))
#print("Elevation gain: {} meters".format(elevation_gain))


In [ ]:
!pip install folium

##Plot map

In [ ]:
import os
import folium

def plot_route(data, route_map, index):

    # Extract coordinates for the route
    coordinates = data['features'][0]['geometry']['coordinates']

    # Add the route polyline to the map
    folium.PolyLine(
        locations=[[coord[1], coord[0]] for coord in coordinates],
        color='blue',
        weight=5,
        opacity=0.7
    ).add_to(route_map)

    # Add start and end markers
    folium.Marker(
        location=[start_point[0], start_point[1]],
        icon=folium.Icon(color='green', icon='flag')
    ).add_to(route_map)
    # Add end markers and popup lable to show the route index
    folium.Marker(
        location=[end_point[0], end_point[1]],
        icon=folium.Icon(color='red', icon='flag'),
        popup=f"Route #{index}"
    ).add_to(route_map)





    # Add label with route index to the midpoint and end of the route

    midpoint_index = len(coordinates) // 2
    midpoint = [coordinates[midpoint_index][1], coordinates[midpoint_index][0]]
    
    folium.map.Marker(
        location=midpoint,
        icon=folium.DivIcon(
            icon_size=(150,36),
            icon_anchor=(75,18),
            html=f'<div style="font-size: 18pt; font-weight:bold; text-align:center; color:navy;">{index}</div>',
        )
    ).add_to(route_map)
    
    
    folium.map.Marker(
        location=[end_point[0], end_point[1]],
        icon=folium.DivIcon(
            icon_size=(150,36),
            icon_anchor=(75,18),
            html=f'<div style="font-size: 24pt; font-weight:bold; text-align:center; color:purple;">{index}</div>',
        )
    ).add_to(route_map)


  

    return route_map

In [ ]:
import os
import folium
def single_plot_route(data):

    # Extract coordinates for the route
    coordinates = data['features'][0]['geometry']['coordinates']

    # Calculate the mid-point for the map center
    mid_point = len(coordinates) // 2
    map_center = [coordinates[mid_point][1], coordinates[mid_point][0]]

    # Create a folium map
    single_route_map = folium.Map(location=map_center, zoom_start=14)

    # Add the route polyline to the map
    folium.PolyLine(
        locations=[[coord[1], coord[0]] for coord in coordinates],
        color='blue',
        weight=5,
        opacity=0.7
    ).add_to(single_route_map)

    # Add start and end markers
    folium.Marker(
        location=[start_point[0], start_point[1]],
        icon=folium.Icon(color='green', icon='flag')
    ).add_to(single_route_map)




    folium.Marker(
        location=[end_point[0], end_point[1]],
        icon=folium.Icon(color='red', icon='flag')
    ).add_to(single_route_map)

  

    return single_route_map



# Time-series of orders per day

In [ ]:
#Time series
#This code ensures that the total number of orders per day is a random number between 20 and 30

import random
from datetime import datetime
import pandas as pd

# define the order times when orders tend to increase
lunch_time = 12
dinner_time = 18

# define the list of order times (in hours), assuming a 24-hour clock
# this one conflicts with the times we use later
# order_times = [11, 11.50, 12, 12.5, 13, 13.5, 14, 14.5, 15, 15.5, 16, 16.5, 17, 17.5, 18, 18.5, 19, 19.5, 20, 20.5, 21]
order_times = [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

# define a dictionary to store the order frequency count for each hour
order_frequency = {}
for t in order_times:
    order_frequency[t] = 0

# define a weighted probability function for each hour of the day
def prob_dist(hour):
    if hour == lunch_time or hour == lunch_time+2:
        return 0.5
    elif hour == dinner_time or hour == dinner_time+1:
        return 0.5
    else:
        return 0.2

# simulate order frequency counts for a specified number of days
num_days = 1
for i in range(num_days):
    total_orders_per_day = random.randint(20, 30)
    orders_placed = 0

    while orders_placed < total_orders_per_day:
        for t in order_times:
            # randomly generate a value between 0 and 1 and check if it is less than the probability distribution for the given hour
            if random.random() < prob_dist(t):
                order_frequency[t] += 1
                orders_placed += 1

                if orders_placed >= total_orders_per_day:
                    break

# print the order frequency for each hour, indicating lunch and dinner times
for t in order_times:
    if t == lunch_time:
        print(f"Lunch hour ({t}): {order_frequency[t]} orders")
    elif t == dinner_time:
        print(f"Dinner hour ({t}): {order_frequency[t]} orders")
    else:
        print(f"Hour {t}: {order_frequency[t]} orders")
print('total_orders_per_day=',total_orders_per_day)
# create a pandas data-frame to store the order frequency for each hour
dt = pd.DataFrame.from_dict(order_frequency, orient='index', columns=['orders'])

# add a column to indicate lunch and dinner times
dt['time'] = [ f"{t}" for t in dt.index]

# print the data-frame
print(dt)

# Calling functions and derive travel information for the time series

Conventional bike average:  0.055-0.077 lb CO2e per mile

Ebike average: 0.046-0.055 lb CO2e per mile.

https://eponline.com/articles/2023/01/13/environmental-impact-of-bikes-and-e-bikes.aspx

In [ ]:
def calculate_emissions(distance, profile):
    distance_in_miles = distance / 1609.344
    emissions_per_mile = 0
    if profile == "car":
      emissions_per_mile = 0.485 # in lbs of CO2
    elif profile == "ebike":
      emissions_per_mile = 0.046 # in lbs of CO2
    else:
      emissions_per_mile = 0.0727525
    emissions = distance_in_miles * emissions_per_mile
    return emissions

In [ ]:
!pip install xlsxwriter

In [ ]:
import time as tm
import datetime
import string
import datetime
import os
import xlsxwriter
import openpyxl
import pandas as pd

output_data = pd.DataFrame(columns=['hour', 'start_lat', 'start_lon', 'end_lat', 'end_lon', 'travel_time', 'travel_distance', 'average_slope'])
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
os.mkdir(os.path.join(os.getcwd(), timestamp)) # creates the directory with the timestamp

#start_lat_lon = (41.71391, -111.83517) #South branch of Domino Pizza
start_point = (41.75312339358314, -111.83486333648153)  # North branch of Domino Pizza

output_data_list = []

date = datetime.date(2022, 1, 1)
nn=0
route_map = folium.Map(location=[start_point[0], start_point[1]], zoom_start=14)

for j in range(len(dt)):
    print('--------------------------------------------------------------')
#    print('n orders for hour', dt.time.iloc[j], 'is=', dt.orders.iloc[j])

    chosen, lat_lon_pairs = choose_locations(dt.orders.iloc[j])
    for i in range(len(lat_lon_pairs)):
        end_point = lat_lon_pairs[i]
        time = datetime.time(int(dt.time.iloc[j]))
        dtime = datetime.datetime.combine(date, time)
        travel_time, travel_distance, slope, elevation_gain, elevation_down, data = get_route_info(start_point, end_point, "cycling-regular")
        tm.sleep(2)
        car_travel_time, car_travel_distance, slopec, car_elevation_gain, car_elevation_down, datac = get_route_info(start_point, end_point, "driving-car")
        tm.sleep(2)
        ebike_travel_time, ebike_travel_distance, slopee, ebike_elevation_gain, ebike_elevation_down, datae = get_route_info(start_point, end_point, "cycling-electric")
        tm.sleep(2)

        route_map = plot_route(data, route_map, nn) #call combined map function
        nn += 1

        # Generate plot and save to "maps" directory
        # route_map = plot_route(data)
        #file_path = f"D:/maps/route_map_{i}.html"
        #route_map.save(file_path)

        # Print the results
        # print(f"From ({start_lat_lon[0]}, {start_lat_lon[1]}) to ({end_lat_lon[0]}, {end_lat_lon[1]})")
        # print("Travel time: {} seconds".format(travel_time))
        # print("Travel distance: {} meters".format(travel_distance))
        # print("Average slope: {}%".format(round(slope*100, 10)))
        # print('')

        car_emissions = calculate_emissions(travel_distance, "car")
        ebike_emissions = calculate_emissions(ebike_travel_distance, "ebike")
        bike_emissions = calculate_emissions(travel_distance, "bike")
        # Add the results to the output_data DataFrame
        output_data_list.append({
           'hour': dt.time.iloc[j],
           'tazid': chosen[i],
           'start_lat': start_point[0],
           'start_lon': start_point[1],
           'end_lat': end_point[0],
           'end_lon': end_point[1],
           'travel_time': travel_time,
           'travel_distance': travel_distance,
           'elevation_gain': elevation_gain,
           'elevation_down': elevation_down,
           'average_slope': round(slope * 100, 10),
           'bike_emissions': bike_emissions,
           'ebike_emissions': ebike_emissions,
           'car_emissions': car_emissions,
           'car_travel_time': car_travel_time,
           'car_travel_distance': car_travel_distance,
           'car_elevation_gain': car_elevation_gain,
           'car_elevation_down': car_elevation_down,
           'car_average_slope': round(slopec * 100, 10),
           'ebike_travel_time': ebike_travel_time,
           'ebike_travel_distance': ebike_travel_distance,
           'ebike_elevation_gain': ebike_elevation_gain,
           'ebike_elevation_down': ebike_elevation_down,
           'ebike_average_slope': slopee,
        })

output_data = pd.concat([output_data, pd.DataFrame(output_data_list)])
#print('******************************************************************')
print(output_data)
# Save the output_data DataFrame as an Excel file
#output_data.to_excel('output_data.xlsx', index=False)

In [ ]:
# Create directory if it doesn't exist
default_dir_name = 'data_15_april_1'
if not os.path.exists(default_dir_name):
    os.makedirs(default_dir_name)

# Get directory name input from user
dir_name = input("Enter directory name for saving excel data (press enter to use default directory '" + default_dir_name + "'): ")

# Get the sheet number input from the user
#sheet_num = input("Enter sheet number for saving the output: ")

# Use default directory name if user input is empty
if not dir_name:
    dir_name = default_dir_name

# Create directory if it doesn't exist
if not os.path.exists(dir_name):
    os.makedirs(dir_name)

# Save the combined map to an HTML file


# Save the combined map to a new HTML file with the timestamp in the filename
path = os.path.join(dir_name, "combined_routes_" + timestamp + ".html")
route_map.save(path)

# Construct the output file path using the input directory name and file name
file_path_data = os.path.join(dir_name, "output_data.xlsx")

# Load the existing Excel file if it exists, otherwise create a new one
try:
    book = openpyxl.load_workbook(file_path_data)
except FileNotFoundError:
    book = openpyxl.Workbook()

# Check if the sheet already exists and delete it if it does
#if 'Sheet' + str(sheet_num) in book.sheetnames:
#    book.remove(book['Sheet' + str(sheet_num)])

# Create a new sheet and write the data frame to it
writer = pd.ExcelWriter(file_path_data, engine='openpyxl')
writer.book = book
output_data.to_excel(writer, sheet_name='Sheet' + str(writer.sheets.keys().__len__()+1), index=True)

# Save the changes and close the writer
writer.save()
writer.close()

# Confirm that the new data has been added to the Excel file in the new sheet
print("The default directory is: " + os.getcwd())
print('Data has been saved to sheet ' + str(writer.sheets.keys().__len__()) + ' in file: ' + file_path_data)
print("The combined_map has been plotted at: " + path)
#print("single maps have been plotted at: " + file_path)

In [ ]:
output_data.shape

In [ ]:
graph_df = output_data.copy()

In [ ]:
output_data = pd.concat([output_data, pd.DataFrame(output_data_list)])
print('******************************************************************')
output_data.columns.values

In [ ]:
output_data['time_diff'] = output_data['car_travel_time'] - output_data['travel_time']

In [ ]:
import matplotlib.pyplot as plt

# Generate scatter plot
fig, ax = plt.subplots()
ax.scatter(output_data['car_travel_time'], output_data['travel_time'], c=['green' if diff >= -300 else 'red' for diff in output_data['time_diff']])

# Set axis labels and title
ax.set_xlabel('Car Travel Time (s)')
ax.set_ylabel('Bike Travel Time (s)')
ax.set_title('Bike vs Car Travel Times')


In [ ]:
import matplotlib.pyplot as plt

# Generate scatter plot
fig, ax = plt.subplots()
ax.scatter(output_data['car_travel_time'], output_data['ebike_travel_time'], c=['green' if diff >= -300 else 'red' for diff in output_data['time_diff']])

# Set axis labels and title
ax.set_xlabel('Car Travel Time (s)')
ax.set_ylabel('Bike Travel Time (s)')
ax.set_title('Bike vs Car Travel Times')


In [ ]:
import matplotlib.pyplot as plt

# Generate scatter plot
fig, ax = plt.subplots()
#ax.scatter(output_data['car_travel_time'], output_data['ebike_travel_time'], c=['green' if diff >= -180 else 'red' for diff in output_data['time_diff']])
import seaborn as sns

sns.barplot(data=output_data[['car_travel_time', 'ebike_travel_time', 'travel_time']])
# Set axis labels and title
ax.set_xlabel('Transportation Type')
ax.set_ylabel('Travel Time (s)')
ax.set_title('Average Travel Times')

plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Select only the columns you want to plot
df_subset = graph_df[['travel_time', 'car_travel_time', 'ebike_travel_time']]

# Plot the data
ax = df_subset.plot(kind='bar', figsize=(10,6))

# Set the plot title and axes labels
ax.set_title("Travel Times by Transportation Type and Route")
ax.set_xlabel("Route")
ax.set_ylabel("Travel Time")

# Show the plot
plt.show()

In [ ]:
output_data

In [ ]:
import matplotlib.pyplot as plt

# Generate scatter plot
fig, ax = plt.subplots()
ax.scatter(output_data['ebike_travel_time'], output_data['travel_time'], c=['green' if diff >= -300 else 'red' for diff in output_data['time_diff']])

# Set axis labels and title
ax.set_xlabel('Ebike Travel Time (s)')
ax.set_ylabel('Bike Travel Time (s)')
ax.set_title('Bike vs Ebike Travel Times')


In [ ]:
import matplotlib.pyplot as plt

# Generate scatter plot
fig, ax = plt.subplots()
ax.scatter(output_data['car_travel_time'], output_data['car_emissions'])

# Set axis labels and title
ax.set_xlabel('Car Travel Time (s)')
ax.set_ylabel('CO2 Emissions (pounds)')
ax.set_title('Car Travel and Emissions')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a new dataframe with the emissions data
emissions_df = output_data[['car_emissions', 'ebike_emissions', 'bike_emissions']]

# Reshape the dataframe to a long format using the `melt()` function
emissions_df = emissions_df.reset_index()
emissions_df = emissions_df.melt(id_vars=['index'], value_vars=['car_emissions', 'ebike_emissions', 'bike_emissions'])
emissions_df.columns = ['index', 'Transportation', 'Emissions']

# Create the grouped bar chart using Seaborn
sns.set_style('whitegrid')
g = sns.catplot(x='Transportation', y='Emissions', data=emissions_df, kind='bar', alpha=0.5, legend=False)

# Set the axis labels and title
g.set_axis_labels('Route', 'Emissions (lb CO2e per mile)')
plt.title('Emissions by Mode of Transportation')

# Add the legend
plt.legend(loc='upper right')

# Show the plot
plt.show()

In [ ]:
!pip install traveltimepy


    sdk = TravelTimeSdk('670bfd71', 'bebf824da7db22390eec6f0b04ccf3c3')

In [ ]:
import requests
import json
import random
import string

def generate_random_id():
    characters = string.ascii_uppercase + string.digits
    return ''.join(random.choice(characters) for _ in range(5))


# Set up your API credentials and endpoint URL
api_id = '670bfd71'
api_key = 'bebf824da7db22390eec6f0b04ccf3c3'
endpoint_url = 'https://api.traveltimeapp.com/v4/time-filter'
all_data = ''


headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'X-Application-Id': '670bfd71',
    'X-Api-Key': 'bebf824da7db22390eec6f0b04ccf3c3'
}

#start_lat_lon = (41.71391, -111.83517) #South branch of Domino Pizza
start_lat_lon = (41.75312339358314, -111.83486333648153)  # North branch of Domino Pizza

locations = [{"id": "Dominoes_North", "coords": {"lat": 41.75312339358314, "lng": -111.83486333648153}}]
data_list = []

for index, row in output_data.iterrows():
    data = {"id": generate_random_id(), "coords": {"lat": row["end_lat"], "lng": row["end_lon"]}}
    data_list.append(data)

departure_times = [
    "2019-01-16T11:00:00Z",
    "2019-01-16T12:00:00Z",
    "2019-01-16T13:00:00Z",
    "2019-01-16T14:00:00Z",
    "2019-01-16T15:00:00Z",
    "2019-01-16T16:00:00Z",
    "2019-01-16T17:00:00Z",
    "2019-01-16T18:00:00Z",
    "2019-01-16T19:00:00Z",
    "2019-01-16T20:00:00Z",
    "2019-01-16T21:00:00Z",
    "2019-01-19T11:00:00Z",
    "2019-01-19T12:00:00Z",
    "2019-01-19T13:00:00Z",
    "2019-01-19T14:00:00Z",
    "2019-01-19T15:00:00Z",
    "2019-01-19T16:00:00Z",
    "2019-01-19T17:00:00Z",
    "2019-01-19T18:00:00Z",
    "2019-01-19T19:00:00Z",
    "2019-01-19T20:00:00Z",
    "2019-01-19T21:00:00Z",
]

responses = []
for departure_time in departure_times:
  body = {
      "locations": locations + data_list,
      "departure_searches": [
          {
              "id": "One-to-many Matrix",
              "departure_location_id": "Dominoes_North",
              "arrival_location_ids": [d["id"] for d in data_list],
              "departure_time": departure_time,
              "travel_time": 3600,
              "properties": ["travel_time", "distance"],
              "transportation": {"type": "driving"}
          }
      ]
  }

  # Make the API call and get the response
  response = requests.post(endpoint_url, headers=headers, json=body)

  # Print the response content (this should be a JSON object)
  print(response.content)

  result_str = response.content
  responses.append(result_str)

In [ ]:
import json
import pandas as pd

# Create a list of dictionaries to represent the data
test_list = []  

for index, i in enumerate(responses):
    # Assuming the string is stored in a variable called "result_str"
    # First, decode the bytes string to a regular string
    result_str = i.decode('utf-8')

    # Load the JSON string into a Python dictionary
    result_dict = json.loads(result_str)

    # Extract the "locations" data from the dictionary
    locations = result_dict['results'][0]['locations']

    for location in locations:
        properties = location['properties'][0]
        data_dict = {'id': location['id'], 'travel_time': properties['travel_time'], 'distance': properties['distance'], 'departure_time': departure_times[index]}
        test_list.append(data_dict)

# Convert the list of dictionaries to a dataframe
df = pd.DataFrame(test_list)

In [ ]:
df.head()

In [ ]:
import calendar

def convert_time(date_str):
  # Extract day of the week
  day_of_week_str = date_str[:10]
  # Convert to day of the week (0 = Monday, 6 = Sunday)
  day_of_week = (datetime.datetime.strptime(day_of_week_str, '%Y-%m-%d').weekday()) % 7
  weekday_name = calendar.day_name[day_of_week]
  # Extract time
  time = date_str[11:16]

  return {'date': weekday_name, 'time': time}

new_columns = df['departure_time'].apply(convert_time).apply(pd.Series)
df = pd.concat([df, new_columns], axis=1)

In [ ]:
df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set the style and size of the plot
sns.set_style('whitegrid')
plt.figure(figsize=(10, 6))

# Create the line plot
sns.lineplot(x='time', y='travel_time', hue='date', data=df)

# Add axis labels and a title
plt.xlabel('Departure Time')
plt.ylabel('Travel Time (seconds)')
plt.title('Travel Time vs Departure Time')

# Rotate the x-axis labels for better readability
plt.xticks(rotation=45)

# Display the plot
plt.show()


In [ ]:
df.shape

In [ ]:
output_data.shape